In [ ]:
!pip install mxnet
!pip install gluonnlp==0.9.1
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-t6lr0wtf/kobert-tokenizer_d93a9e1d0ea84e85ab9024fd60bbe1cf
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-t6lr0wtf/kobert-tokenizer_d93a9e1d0ea84e85ab9024fd60bbe1cf
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/'Colab Notebooks'/KHUDA_TOY
!ls
import sys; sys.path.append('..') # add project root to the python path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/KHUDA_TOY
result1.xlsx  result2.xlsx


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('result2.xlsx')

chatbot_data = chatbot_data.dropna()
chatbot_data['Label']=chatbot_data['Label'].astype(int)

In [ ]:
chatbot_data.sample(n=10)

,Label,댓글 내용
9082,0,그럼 lg는 또 답이 없는 거냐?
9492,0,카더라 통신으로 뭘하겠다는건지...
9359,0,이정도면 더 볼것도 없네
29803,1,좋아죽겠다 진짜로ㅠㅠㅠㅠㅠㅠㅠ!!!!!!!!!
9730,0,법을 바꿔 그니까 닭대갈 색끼들아
25143,0,김해 부산사람은 경찰 무서워서 살겠나
7920,0,증인보호를 왜 이따위로 합니까
19820,1,백선생님 응원합니다...
31997,1,고현정 너무 멋진배우. 여자로도 엄마로도 배우로도 사람으로 모두.
12108,0,미치겠다..


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
chatbot_data.loc[(chatbot_data['댓글 내용'] == "부정"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['댓글 내용'] == "긍정"), 'Emotion'] = 1  #놀람 => 1

data_list = []
for q, label in zip(chatbot_data['댓글 내용'], chatbot_data['Label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[4000])


['언니 동생으로 부르는게 맞는 일인가요..??', '0']
[' 충전할때하다 불안해서 폰끄고 배란다에서 충전함', '0']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

25296
8432


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
batch_size = 64
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
max_len = 64

In [ ]:
# ★
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
data_train[0]

(array([   2,  517, 7673, 7682, 5770, 7835, 3574, 5865, 3256, 7782, 5702,
        3085, 5355,  905, 5931,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-21-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/396 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7546115517616272 train acc 0.5
epoch 1 batch id 201 loss 0.29642778635025024 train acc 0.8059701492537313
epoch 1 train acc 0.8536537247474747


<ipython-input-21-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/132 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
#토큰화
tok = tokenizer.tokenize
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("부정적인 감정이")
            elif np.argmax(logits) == 1:
                test_eval.append("긍정적인 감정이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

NameError: ignored

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 더워서 좋아
[ 0.22993438 -0.02416631]
>> 입력하신 내용에서 부정적인 감정이 느껴집니다.




KeyboardInterrupt: ignored

In [ ]:
tok = tokenizer.tokenize
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            # print(logits)
            if np.argmax(logits) == 0:
                return 0
            elif np.argmax(logits) == 1:
                return 1

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

URL = input("긍/부정을 확인할 유튜브 링크 : ")

driver = webdriver.Chrome(ChromeDriverManager().install())
#크롤링할 유튜브 불러오기
driver.get(URL)
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)

#웹페이지 끝까지 스크롤하기
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

#유튜브 팝업 닫기
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass

#댓글 가져오기
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

comment_list = soup.select("yt-formatted-string#content-text")
comment_final = []
count = 0
result = 0

for i in range(len(comment_list)):
    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용
    count+=1

for sentence in comment_final :
    result += predict(sentence)

result/=count
if count>=0.5 :
  print(f"내용에 {result/count*100}%가 긍정적입니다")
else :
  print(f"내용에 {100-(result/count*100)}%가 부정적입니다")

In [ ]:
comment_final = ['상황 정리1. 최소라가 앞 순서 모델 분들의 포즈나 제스처 등을 반복적으로 카피함2. 모델 분들 닥쳐로 열 올라서 카피 못 하게 촬영 순서 1번 시킴 (1번은 먼저 하는 대신 좋은 장비 선택 가능)3. 최소라 열 받음4. 다리에 쥐 났다는 핑계로 내려와서는 뒷 순서에서 1번 특혜인 좋은 장비 + 앞 순서 포즈 또 카피', '처음에 걸어 나올때 피지컬 천상계다 진짜ㅋㅋㅋ', '지금 최소라 넘사인거 인정하지만 방송당시엔 고소현이 최소라보다 화보 월등하게 잘찍은것도 인정해야함. 존재감 1도 없다가 3연승으로 포텐 터졌을때 최고였음.(후반부 김진경 포텐 터지는거, 초중반 고소현 캐리가 꿀잼이였음)', '최소라 처음에 드레스입고 걸어오는데 와... 완전 찰떡이고 올빽 포니테일도 너무 잘어울림 탑인 이유가 있는듯 너무 예뻐', '최소라 타고난건 그냥 모델 그 자체.. 멋있어..', '최소라가 어딜 가든 적이 있을 스타일이다', '난 저때 소라가 귀여워서 좋음ㅋㅋㅋㅋㅋㅋㅋㅋㅋ미워할 수 없는 매력임', '이거 재밌게봤는데ㅋㅋㅋㅋ 이제와서 최소라 무조건적으로 실드치는 사람 많은데 이 방송할때는 비호감으로 행동한건 맞음.. 이기적이기도 했고ㅋ 근데 그 후에 최소라가 자기가 한 행동에 대해서 사과할 부분 사과했고 반성하는 모습도 보였음.. 자기잘못 인정 못하는 사람도 많은데 인정할건 인정하는거 멋있는거 아닌가? 성공에 악착같이 매달리다보면 나올수있는 행동이기도 하고 그거갖고 지금까지 인성어쩌고하는건 좀ㅋ 사람이 점점 성숙해지면서 바뀌는 경우도 많은데... 그리고 다시보니 최소라 성공할만하다고 생각이 들었음ㅋㅋㅋ 열정이 대단', '이거 보고 넘 재밌어서 결제하고 도수코3 다봤는데..첨부터 끝까지 보면 시즌3 모델들 다 성격 장난없음ㅋㅋㅋㅋ나라면 최소라는 오히려 성격이 너무 예측가능하고 일관적이라 적당히 거리두고 싶은 정돈데,고소현님 만만치 않더라.. 너무 어려운 성격.예측 불가능.. 약간 중학생 때 여자들끼리 기싸움하던 아픈 추억들이 생각나는 성격이심ㅠ', '난 이 사람 팬도 아니고 어쩌다 들어와 보니 예전에 본 방송이네요.중간까지 보다 말았던건지 결과도 잊었네요.지금은 외국살지만 저 때 한국에서 참 재미있는 방송 많이 했었어요.저 시절에 방영한 트럼프 대통령 나온 어프렌티스도 재밌게 본 리얼리티프로그램 중 한 편인데 그때 보면서 방송으로 다 보이고 하는 계략 모략쓰는 사람들이 결국 심사에서 뽑혀서 그 중에서 또 우승까지도 하는걸보고 너무 황당했었는데 오히려 좋은 의미로 정치적이라며 칭찬받더라구요.시청자들도 좋아하구요.우리나라의 가치관으로는 인성에 문제있어보이는 거짓말하는 사람을 뽑는다는건 있을수없는 일 같았는데..하지만 나이들고 지나서 생각하니 목표를 위해서 타인을 밟고 일어서고 수단과 방법을 가리지 않았다는 것보단,결국엔 좋은 결과물을 낸 실력자가 뽑히더란겁니다.떨어진 사람이 성인군자인것도 아니었고 야비해 보였던 사람이 끝까지 야비한 방법으로 올라간것도 아니고 찬스를 잡고, 좋은 아이디어는 적절히 자기것으로 만들고 어필했더라구요.최소라도 저러다 도태되었으면 그냥 재수없는 애로 끝났겠지만 좋은 결과를 내고 있으니 무작정 인성파탄난 일인으로 매도하기보단 앞으로 여러 상황이 닥칠 때 지금 왜 이걸 하고 있는지에 포커스를 맞추고 덜 중요한 부분은 무시할수있는 판단력과 무엇보다 실력을 쌓고 그 실력을 십분 보여줄수있는 능력을 키우는게 교훈인듯 해요.살아가다보면 더하든지 덜하든지 비슷한 상황들과 마주치게 되지요.그냥 욕하고 지나가기보단 나라면 어떻게 할까..한번 생각해보는것도 좋을거같아요.', '지금은 진짜 탑모델이지..', '1. 최소라가 다른 모델들 포즈를 계속 따라함 (예) 스퀴즈 광고 찍을 당시 이나현의 컵 구기는 아이디어를 훔치고 찍은 다음 이나현한테 그거 안좋다고 했고 그걸로 호평받음2.그래서 전 미션 우승자가 순서 정할 수 있었는데 최소라를 맨처음으로 둠 (대신 좋은 장비)3. 갑자기 쥐났다고 하면서 위에서 시간 엄청 잡아먹고 그냥 내려옴4. 자기는 쉬면서 다른 모델들 하는 거 다 봄 그리고 따라함5. 최한빛에게만 있는 총을 자기가 씀', '화장지우니깐 진짜 순둥순둥 소녀네', '성공한 사람은 뭐 무조건 칭찬만 받아야하나? 갠적으로 너무 오래된 일 가지고 죽어라 깔 필요까진 없다고 느끼긴 하는데 그렇다고 반대로 과거라고 해서 다 없는일인척, 성공했다고 해서 과오까지 다 쉴드쳐줄 필욘 없다고 생각함 성공과 인성은 별개의 문젠데 왜 그걸 꼭 비례하는것처럼 묶으려고 하는지 그거자체가 이해가 안됨ㅋㅋ 그리고 성공한 사람 중에 편법 남에게 피해 안끼치고 성공한 사람도 많은데 꼭 저렇게 살아야 성공한다 ~ 라고 일반화하는것도 굉장히 위험하다고 생각함. 이런 케이스가 오히려 튀니까 사람들이 인상깊게 기억하는거 같은데.', '지금은 너무 넘사벽 모델..ㅋㅋㅋㅋㅋ', '이때도피부짱좋다..역시 타고났네 ㄷㄷ', '이길려면 착하면 안 되는 업계 중 하나가 모델업계라고 생각함모델 업계는 실력도 실력이지만, 그만한 멘탈도 필요한 듯', '솔직히 둘다 웃김ㅋㅋㅋㅋㅋㅋ 자기 1번으로 시켜서 기분 안 좋은거 티 다 내놓고 그냥 쳐다본거라고 하는 최소라나 당연히 1번 줬으니까 기분 나쁠만한데 왜 째려보냐고 물어보는 고소현이나 ㅋㅋㅋ', '시즌 3 전부 다 본 사람이면 절대 최소라 쉴드 못침', '넘사벽이든뭐든 저런사람이 되진말자  인성이 글렀다', '저기 있잖아요저당시에 최소라가 잘못을 하긴 했는데 최고의 배드걸 최소라라뇨..그냥 다 어그로때매 제목을 이따구로 적어놓으신지?상황정리도수코에선 모든 모델들이 다 나쁘게 나옵니다최소라가 카피한게 많긴 한데요나머지 멤버들도 똑같아요똑같이 최소라 욕할땐 언제고 자기들끼리도 싸워..솔직히 제 생각으로는 도수코 자체가 쬐금 이상한 면이 있는건지,아니면 한 집에 가둬놓고 거기에서 지내라 하니 스트레스 받아서 그런건지..;;;제목 어그로 이딴식으로 하지마요조회수올리려고 왜 제목을 이렇게 잡아요??', '도수코보면 전체적으로 성격이 대단해서 ,, 딱히 최소라가 나쁘지도 않음 오히려 순수하면 모를까 그래도 이 장면에선 잘못한거 맞고', '0:29 웃겨 진쫘앙~ㅋㅋㅋ', '이쁘긴 이쁘다 소라 너무 이쁨', '저때 잘못한 건 맞지만 지금은 7년이 지난 시점이니 이 시절의 태도로 인성을 운운하는 건 바람직하지는 않다고 봄.. 사람은 안 변한다고 하지만 스스로를 돌아봤을 때 과연 1~2년 전의 나와 지금의 내가 같다고 볼 수 있나? 아닐 거임. 지금 하는 행동에서 문제가 될 게 보인다면 그걸 지적하는 건 맞겠지만.', '최소라 볼때마다 드는 생각이지만 진짜 탑이 되려면 저정도는 감수해야하는건가라는 생각이 듬ㅠㅠ', '보면요 인상에서 느껴져요 최소라 똑부러지고 착한게 느껴집니다.', '시즌 3 내내 밥맛이었는데 이거보다 더 기분 나쁜 부분은 시즌 4. 본인들이 서브로 주인공들이 잘 할 수 있도록 도와줘야 하는 입장인데 자기가 메인인 줄 알고 끼부리는 부분 진짜 너무 별로 였음. 더 어이 없는 건 신애랑 찍을 떄는 독하게 굴더니  혜아랑 찍을 때는 아는 사이라고 배려 해주는 모습이 공평하지 않아 보여서 더 마음에 안 들었음. 얘는 실력이 아무리 좋아도 칭찬해주고 싶지 않은 그런 인성을 가진 거 같음.', '안보면 후회할정도는 아닌데 수위는 바바티비가 ㅋㅋㅋ 진짜 지리긴하더라', '원래 오리지널 아메리칸넥스트모델에서도 싸가지없는애들이 성적이 좋은경우가 많아. 실제로 나오미캠밸 인성도 잘알려져있고. 저세계자체가 좀 싸가지없어야 성공할수있는 세계인거같음. 워낙치열하니까', '모델 최소라는 진짜 좋은 모델인데 사람 최소라는 인성이란게 터져버림 ㅇㅇ', '0:16 왜 째려보니?0:19 왜 째려봐! 라고 했다엇갈리는 진술0:19  문장이 더 나쁘게 과장되고 어투도 강하고 나쁘게 바뀐 상태로 인터뷰', '이거 보면 최소라가 뭐 어떠하다 하는데 뒤로 갈수록 고소현도 만만치가 않았음 ㅋㅋㅋㅋㅋ', '최고의복수 내가잘살고있음됨', '난 최소라가 누군지도 몰랐는데 뭔 또 탑모델이여..? 전혀 몰랐음.....', '소라 언니 화이팅', '유스케보고 왔는데 이영상보니까 확 깨네', '저렇게 자신만만하게 해놓고 포즈, 소품 등등 맘에 안들었는지 다리 쥐났다고 다른사람들도 다 똑같이 위에서 밑으로 내려왔는데 혼자만, 딱 지 혼자만 다리 쥐났네 어쩌네 하면서 결국 저래놓고도 맨 마지막으로 사진찍음ㅋㅋ 다른 사람한테만 지정된 소품 몰래 차고와서는ㅋㅋ 이러니 밉다는 소리를 듣지..', '잘 참아냈어 지금은 성공해서 너무 훌륭한 모델이 잖아~?애당초 여자들 시기질투로 넘어갈 재목이 아니였어~아주 칭찬해', '근데 왜케 최소라한테 정이 가지;;캐릭터 ㅈㄴ 웃김ㅋㅋㅋ친구하면 재밌을 것 같음 솔직히', '이게 해외에이전시가 껴잇어서 최소라가 우승한거지 한국에서만 햇으면 여연희나 다른애가 우승햇을거임... 바로 해외에이전시로 나가는걸로 계약하고 하니까 거기 의견이 반영이 엄청 됐을거임 지금은 물론 멋지고 대단한데 프로그램 당시에 욕먹은건 이해못하는 사람 없었을거임', '근데 저날 최소라 드레스 ㅈㄴ예쁘다 ㄷㄷ', '도수코3 애청자로서  고소현이 더 싫었음.  뒷담화 너무 심해서..  여우짓하는 애들 존많이였는데 최소라는 뒷담화 없이 걍 지할일 열심히해서 멋졌음. 여론도 뒤편으로 갈수록 최소라 응원하는 사람많았고', '그땐 몰랐겠지,,,,, 최소라가 성공할지', '인성은 직접 겪어봐야 판단하는거고...최소라 커리어는 탑이지만 그닥 팬은 아님 그치만 방송국놈듷이 각잡고 악편한걸 왜 그대로 받아들여....도수코 특징이 사람 한명 빌런 만드는거임...쫄쫄 굶으면서 서바이벌 하는 애들이 얼마나 예민하겠냐 나이도 어린애들임 최소라 저때 20대 극초반 아닌가', '이때 본방본사람인데 걍봐도진짜 화나는 부분많아서ㅋㅋㅋ악마의편집도아님ㅋㅋㅋ있는그대롴ㅋㅋㅋ이거때매 최소라 폭행사건터짐 안티가 맘에안든다고 팼음 그때진짜심각했음 이시즌에 인성안좋은애들진짜 많이나왔는데 고소현도 몰이해서 한사람 싫어하기 쪽주기 많이했고ㅋㅋㅋ이름기억안나는데 실력도안되면서 인터뷰때마다 남 비웃어서 욕먹은애도있고 대놓고 하상백이 탑모델에서 떨어지면좋겠다고 그럼ㅋㅋㅋ', '솔직히 이렇게 성공하려면 독해야 하겠지..그래서 성공하고 유명한 연옌중에 우리가 모를 뒷사정같은거 많을듯 인성 하자나고 뭐.. 얘는 과정이 영상으로 남겨져서 그렇지 ㅋㅋㅋㅋㅋㅋ 솔직히 물만먹고 2주...정도면 진짜 지 자신 갉아먹을 정도로 독해야함', '넘사벽이든 ㅋㅋㅋㅋㅋㅋ 모델들 포즈 카피해서 포즈 취하면 누가 기분 안나쁨 ? ㅋㅋㅋㅋㅋㅋㅋ', '최소라 세계 5위 모델되고나서 욕하지마라고 하는 팬들 웃김 ㅋㅋㅋ 세계 1위면 아주 발가락도 빨아야할듯? 인성썩은건 사실이잖아 성공한것도 사실이고', '프로그램이 갈등조성하는듯...순서정하는거나 의상정하는거나 청소정하는거나...저렇게하면 사이 안좋은사람은 더 안좋아지고 갈등만 생기지않나 ..', '도수코 다 보면 한회볼때마다 최애랑 싫은사람이 바뀜 ㅋㅋㅋㅋ 끝까지 정주행함서 솔직히 한명쯤 다 싫어해봤음', '얄밉긴 해도 어쩌겠음 저게 현실이고최소라도 저때 많이 어렸음본인도 나중에 저때 왜그랬는지 모르겠다며 사과함', '최소라만 보면 인성쓰레기 인데여기다른 모델 보면 인성쓰레기 더 있음 훨씬많음', '왜 째려보냐 라고 먼저 말하는게 깡ㄴ ㅕㄴ들이 주로 먼저 던지면서 시비거는말  아닌가요?상대방이 개인적으로 그때 기분이 안좋은 상황일수도 있는건데', '지기 싫었지만 다리에 피가 안통해서 결국 맨마지막에 찍었네ㅋㅋㅋㅋ 근데 맨마지막은 또 잘 내려가던데ㅎㅎ', '뜬금 없지만 나만 고소현 안경벗고 화장하면 보라 닮은것 같나...??', '돈을 잘벌든 뭘 하든 인성이 저러면  전혀 멋있지도 부러워보이지도 않는데', '피해의식,컴플렉스로 가득찬 저 표정과 말투.', '초소라 같은 저런 사람들 때문에무서워서 모델 하겠다는 사람이 있을까?ㅋㅋ', '기분나쁜건 방송에 잘 보이고 싶고 여기저기서 욕먹는거 싫어하거나 원래 웃는 성격이면 저기서도 웃을수 있는데 최소라 성격이면 당연히 불만은 있겠지ㅋㅋㅋ 그럼 뭐 저기서 아니예요 언니 하면서 웃어야함? 저 사람들은 기회 잡으러 나온거지 방송타려고 나온게 아님 난 소현님이 저렇게 왜 째려보지? 하는게 난 이해가 안감저 말을 왜함 불난집에 대놓고 기름붙는 꼴이고 카메라는 돌아가는데 ㅋㅋㅋㅋ 그냥 싫은거임 ㅈㄴ 싫은데 나이 많으니까 이런 말 하면 그냥 유교마인드로 대충 넘어가니까ㅋㅋ 누가 저 상황에 안좋은거 주고 째려본다고 뭐라해 무슨 잘못한것도 아니고 직장 상사도 아니고 제발 사람 성격을 숨기라고 강요좀 하지마 착한게 당연한게 아님', '정치질ㅋㅋㅋㅋㅋ', '근데 저런 깡이 읍음 어떠케 동양인으로 탑모델로 갈 수 있었을까 생각을 해봐 . 해외에서 동양모델 잘 안쓰는데 쓰더라고 중국인정도 쓰지 탑모델이 되기까지 분명 까이고 인종차별 경험했을그다 관계에 있어선 안좋을수 있겠으나 지금 저기선 모두가 경쟁사이므로 저런 깡이 없으믄 힘듬', "예전엔 누근 한명이 얄미웠는데, 이제 보니 삼삼오오 편모여 이 세세한 티테일 니가 따라했네마네 했던 누구들도 썩 이미지가ㅎㅎ 어차피 돌아가는 방식이 룰없는 '개판'이라면, 그 판의 호구는 다수가 되야지. 왜 특정 소수이겠. 외국같앗음 '소수 인종'차별로 난리날 사안. 진짜 문제됐음 그 판의 룰 지정자인 관리자가 직접 나서, 관리 통솔했겠지.. 냅둘땐 큰문제라고 할수없기에 냅두는거고..", '그래도 최소라가 대성공햇지...', '진짜 최소라는 잘나가도 사람으로써 인성이;;', '최소라 존예ㅎㅎ', '최소라가 유독 더 인성논란이 많이 떠서 그런거지 이민정,이나현에 비하면 최소라는 착한거지 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ걔들은 탈락할 때까지 지랄했고 최소라 마지막에는 그래도 좀 고쳐먹었던데', '그래도 최고야...', '최소라 지금보다 저때보니까 진짜 통통했네', '화장의 힘!', '성공했지만 이에피소드의 최소라행동은 잘못한거 맞음', '얘 잘되지 않았으면 지금 유튭에서 정하은이랑 똑같은취급 받고있을듯ㅋㅋㅋㅋㅋㅋㅋ', '이 동영상하나로 그사람 성격에대해 다 안다는듯이 말하는게 너무 웃김 ..자기들 생활이 영상으로 안찍혀서 그렇지 솔직히 우리가 살아오면서 얄밉게행동하거나 정말 보기 안 좋은 행동 한 건 많을거임 ..그런 행동들로 내 인성이 누군가에의해 결정 된 다는게 사실 억울하고 기분나쁘지않나?.. 남에대해 1도모르면서 뒤에서 다 안다는듯이 말하는애들이 제일 꼴 보기싫다 ㅋㅋ', '아 ~ 여기서 인성거리면서 최소라 까는 댓글 다는 건 인성이 좋아서 하는 행동이구나?사람이라면 누구나 여우같은 본성이 있는 게 정상이다. 고소현도 최소라도 둘다 여우짓 하는데 어떤 면에서 여우같은 행동을 하는지 정도가 다른 것 뿐이고, 서바이벌 이기러면 어쩔 수 없는 인간적 본능 아닌가?자기들 인성은 대체 얼마나 좋길레 남의 인성을 논해?', '최소라도 인성 터졌지만 저때 모델들 보면 진짜 나머지도 다 싸가지없고 인성터졌음 그나저나 지금은.....ㄷㄷ', '탑모델  되려면 저런 패기가 있어야지 촬영할때도 당당한 표정 다양한 표정이 나오는거지 겸손하고 그래봐라 ㅋㅋㅋ지보다 언니인 사람들 사이에서 주눅들어 어디 포즈나 제대로 잡겠냐 ㅋㅋㅋ  내가  짱이고 다 들러리란 생각을 가지고 있어야 당당해지는거야 무슨 회사도 아니고 백프로 경쟁구도인   상황에서 겸손 인성이  무슨 소용인데', '방송보면 오히려 최소라는 진짜 잘찍고 싶어서 승부욕에 불타서 뻔히 눈에 보이는 꾀를 부리는데, 저기 나오는 소현? 저 여자를 비롯한 몇명은 촬영 안할때 쓸데없는 일찐짓하면서 애들 기 누르고 다녔음 ㅋ 최소라가 잘 될수밖에 없어. 모델일에 욕심이 많으니까.', '도수코 악편심한거 모르나 ㅜ', '탑모델이건 뭐건.. 인성 별로', '인성을 왜 찾아요 서바이벌인데탑 찍을려면 지기 싫은 악바리같은 마음이 있어야지 10대 20대 어릴때 그런 근성으로 아무나 못함. 떡잎부터 다른 거어차피 패션분야 하나의 소재나 아이템으로 돌고도는 거 누가 더 개성있게 잘살리냐의 문제 그게 실력.', '썸네일 잘못 봐서 최소라 고소함 으로 봤어.....', '소현 저사람은 다른 사람들이랑 참 많이 싸우네', '쟤를 보면 남것 따라고 남무시해야 성공할수있구나 라는 생각이듬', '둘의 기싸움인데 왜 내㉮ 쫄지??', '모델이 꼭 인간적이고 착해야 되냐?? ㅋㅋ 그리고 사람 마다 상황마다 다 틀리지? 미디어에 나온 모습으로 어쩌고 저짜고 하는 인간들 참~', '요즘 유키즈 나와서 최소라 인기 올라갔는데.. 솔직히 .. 이때 욕엄청먹어서 테러 당하고 그랬었음;;ㅋㅋㅋ 솔직히 다시 봐도.. 바디 포지션은 좋은데.. 화보 모델은 모르겠슴..', '목소리 넘 달라졋네', '저정도 성깔 되니까 외국가서 기 안죽고 하는거지 난 멋있다 저런성격 부럽고', '최소라 이거 다음시즌에서 도수코 시즌 1.2.3 우승자들 나와서 촬영 도와주는거 하는데 , 아주 자기가 더 주인공이야 ㅋㅋㅋㅋ 이지민이랑 진정선은 자기가 엑스트라로서 도전자들 더 돋보이게 해주려고 노력하는데 혼자 도전자들 기죽이고 자기가 더 돋보이게 사진찍고 인터뷰에서도 자기가 전시즌1위인데 잘 못하는 모습 보이면 어떡하겠냐고 ... 완전 지들 부른 취지도 모르는 멍충인가 싶었음', '고소현 약간  김연경 닮았어', '잘안됐으면 시즌4 정하은처럼 지금도 내내 영상보는 모든사람들한테 욕먹고있을듯ㅋㅋㅋㅋ 뭐 근데 나도 결과가 중요하다고 생각해서^^ 독하니까 살아남고 올라가는거지뭐', '이래야 성공하나보당', '인성 뭣같은 사람이여도 성공하면 다 받아줌^^', '도수코 나오는 기준이 도대체 뭐지,,인성 쓰레기들만 모으는건가 김진경 빼고 다 인성 뭐같았던 걸로 기억하는데', '최소라 커리어라면 악플 다 고소할 듯한데 yg케이플이 일을 안하는건가?', '사람들 최소라 욕하는데, 저런 애들이 활동도 편함왜? 최소라 못지않게  저것보다 성격 더 한애들 많으니깐. 다들 모델계 모델 아니여도 알잖아요', '인간승리', '그놈의 인성... 그걸로 먹고 사는 사람도 아니고 ㅋㅋㅋ 인성 좀 그만 찾으세요', '최소라 인성 성격 진짜 별로다', '지금은 제일 유명한모델중 하나 ㅋㅋ ㅆㅆㅆㅅㅌㅊ', '누가봐도 째려봄 ㅋㅋㅋㅋ', '저거 다 진짜 인성아님도수코가 편집을 완전 쓰레기 따위로한거임', '최소라가 인성이 좀 그렇긴한데 그럼 정하은이랑 이나현은 얼마나 쓰레기인짘ㅋㅋ', '최소라는 눈치가 없는 애지 악랄한 애는 절대 아님', '??????? 영상이 6년 전 영상인데 썸넬에 실화냐가 있음... 실화냐 유행한지 6년이나 됐음??? 막 그때 타겟인가 쨋든 어떤 남돌이 아련하게 이거 실화냐... 하면서 노래 부른게 6년 전이라고???', '저거끝나고 ㅈㄴ단체 이지매 까이자나여ㅋㄱ 버스안에서 소라보면서 아 죽이고싶다 요러면서 ㄷㄷㄷㄷ', '고소현이나 최소라나 인성 막장인 애들임 ㅋㅋㅋ한명만 욕할거없슴 ㅋㅋ', '진짜 ㅋㅋㅋ개비호감', '저거몇화에요??', '와 친구들아 고소현 이대  컴공과인거 나만 이제 안거야?검ㅈ색해버거 ㅈㄴ널람', '최소라 좋았는데 이 양상 보니까 좀 깨긴한다...', '둘다가정교육이문제야', '고소현은 어디 안끼는데 없어?', '??? 누가 쉴드 치고 있음??', '저거 어차피 다 대본이라는데 왜 님들이 극대노함', '고소현이 쌈닭이였네.이애저애하고 게다가 방송찍는거알면서 저정도면..실제론  성격이 어떻겠냐?', '왜들 그러지 난 저 인성 맘에들던데', '최소라쟤도진짜싸가지다 지만아네항상도수코4나왔을때도 다른애들도와줘찍어야지 지가 제일열심히찍더만', '욕한들 뭐하겠나 루이비통 캠페인 하나로 다른애들 압살하는데', '여기 나온 사람들 다 노답이다..', '최소라를 욕하는건 아니지만 탑이될려면 저래야한다 라는 댓글은 이해가안되네', '유퀴즈에 나온 최소라가 이분맞음?', '고소현 질투 -> 최소라는 피차 고소현이 경쟁상대도 아녔고 자기갈 길 감', '지들이 최소라같은 애들한테 당해봐ㅋㅋㅋ바로 빼액빼액 거리지 게다가 빌런이 엄청 성공까지하면 인터넷에 다 적을꺼면서 지들일아니라고 쉴드오지게들친다', '뭘카피야 소품.컨셉.포즈 다른 모델들도 그들만의 시그니처고 자신만의 아이디어야???걔네도 모방이고 변형이잖아 근데 비슷하다고 카피라고??  사람 생각하는거 다 똑같아 같은포즈 나올수밖에없지 컨셉이 똑같은데', '님들 최소라든 정하은이든 방송은 다 연출 대본임.', '모델이 자기관리 잘하고 프로페셔널하고 그거면 된거지뭐 난 최소라 좋게봄 모든게 마찬가지겠지만 저런 직업군들은 특히나 기 약하고 성격소심한것보단 차라리 기쎄고 당돌한게 훨씬 더 나음', '0:160:19말이 왜곡됨', '모델 왜해?그게멋있어? ㅠㅠ', '최소라 시즌3에거 거의 왕따였음', '0:40  순의 조정줌.', '최소라 욕쳐먹을만함 ㅋ', '뱃걸~ 앤드 굿걸~', '최소라도 최소란데 고소현이 더 싫다. 지적질 뒷담화 작렬 매사 사람공격적인 비아냥 ..그냥 뭐 넘어가는게 없다.미우면 밉다 자꾸 욕하는데 나라도 싫다 .어떻게 이번 편엔 잘한다 잘한다 서로 응원하는 인간들이 없냐..보기만 해도 삶이 지치고 피곤하다...', '여기서 지금 탑모델이라는 얘기는 왜함?ㅋㅋ 이재용도 삼성 부사장이니 덕질하셈ㅇㅇ', '솔까 포즈많이 베끼기는 하는데.. 그러면서 배우는거지 뭐', '편집된 화면만 보고 진실을 알 수 없다 생각...요즘 유난히 이런 내용 뜨는 이유는?', '연기 너무 티난다', '얘너무시러진짜', '0:16 열등감', '성공해도 싫음.', '인성이 별로네..최소라', '둘다 진짜 얄미운스타일최소라는 그래도 좀 싫진않고 뻔뻔한데고소현은 다른편보면 인성에 문제있는듯', '고소현  뭐임', '그냥 둘 다 별론데', '왜 싫음?최소라?', '최소라 겁나좋든데', '응 아니야 여러분 도수코 12회랑 13화 보고 오세요 반성 많이 하고 고치려고 하고 있다구요ㅡㅡ', '근데 저게 나쁜거임?? 화보촬영 경쟁이지 창의력포즈 선발대회도 아니고ㅋㅋ 참고해서 내가 가장 잘할수있는걸 해야지', '여기 댓글들보니까 진짜 막힌사람들 많네요 ...ㅋㅋㅋㅋ솔직히 영상하나로 인성을 판단하는거부터가 웃기지않나요? 도수코3 다본사람으로서 물론 최소라가 잘 못 한 부분들있지만 옛날영상에서도 자기가 했던 행동들 다미안하다고했고 (당사자한테) 심지어 안티팬한테 공격당하면서도 뉘우치는 모습을 보이는사람한테 아직까지도 이영상만보고 인성 운운하면서 트집잡고 욕하는게 웃기네요 ㅋㅋㅋㅋㅋ님들은 잘못안하고사나요? 다들 십선비들인가? 사람들은 누구나 후회할짓 하고 살고 용서도 해줄수있어야한다고 생각함 ...최소라가 누구를 상해입힌것도 아니고 그냥 욕심에서 나온 행동들인데 그걸가지고 아직까지도 우려먹는 한국인 네티즌들반성하는모습도보였는데도 인성을 이영상하나로 아직까지도 단정짓는 사람들님들도 생활하는 일상 리얼리티로 촬영하면 인성갑일듯요 남들인성 판단하기전에 자기들부터 돌아보길', '이거가지고 인성을 논하네 지금 진짜 탑모델에 진짜 미친노력 하고있고 다른영상들이랑 도수코 후반갈수록 진짜 착한모습만보여주는데 누가 누굴 까ㅋㅋ 소라언니보다 인성 개터진애들 개많음', '0:20 0:13', '헐.  . 좀 깬다;;', '왜난마음에들지ㅋㅋㅋ아 진짜예쁘네', '둘다 ㅂㄹ ㅋㅋ', '저런인성으로 성공하면뭐하냐. 반쪽짜리 성공인데', 'ㅂrㅂrㅌIㅂI 홍보좀 그만해라 ㅡㅡ 그래서 어디다 검색한다고?', '고소현 듣보;;']

In [ ]:
count = 0
result = 0
for sentence in comment_final :
    result += predict(sentence)
    count+=1

result/=count
if (result/count)>=0.5 :
  print(f"내용에 {result/count*100}%가 긍정적입니다")
else :
  print(f"내용에 {100-result/count*100}%가 부정적입니다")